##Frechet Inception Distance

In [ ]:
import tensorflow
import os
import numpy as np
import cv2
import random
from scipy.linalg import sqrtm
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
import keras 
from PIL import Image
import imageio
from skimage import img_as_ubyte, io
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from matplotlib.image import imread
import random

In [ ]:
!unzip '/content/mparticles.zip' -d '/content/'

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders  # or import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio('/content/mparticles', output='/content/mparticles-split', seed=1337, ratio=(.7, .15, .15), group_prefix=None) # default values

In [ ]:
g = keras.models.load_model('/content/cgan-particles.h5')

In [ ]:
x = np.random.randn(100 * 2500)
x = x.reshape(2500, 100)
labels = np.asarray([x for _ in range(2500) for x in range(2,3)])

images  = g.predict([x,labels]) 
images = (images+1)/2 * 255

In [ ]:
i = 0
for image in images:
    i += 1
    plt.figure(figsize = (8,1))
    # plt.imshow(image.reshape(48,48), cmap='gray')
    # plt.show()
    img_name = f'ss-cgan-{i}.png'
    imageio.imwrite('/content/gan-generated/SS/'+img_name, image)
  

In [ ]:
categories = ['CS', 'MC', 'SS']
# categories = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
data_directory = '/content/mparticles'


original_data = []

def load_original_data():
    for category in categories:
        path = os.path.join(data_directory, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2. IMREAD_GRAYSCALE)
                resized_array = cv2.resize(img_array, (48,48))
                original_data.append([resized_array, class_num])
            except Exception as e:
                pass

In [ ]:
load_original_data()

In [ ]:
import random
random.shuffle(original_data)

x_original = []
y_original = []

for features,label in original_data: 
    x_original.append(features)
    y_original.append(label)
    
x_original = np.array(x_original).reshape(-1, 48, 48, 1)
y_original = np.array(y_original).reshape(-1)

In [ ]:
original_particles = x_original
original_CS = x_original[y_original==0]
original_MC = x_original[y_original==1]
original_SS = x_original[y_original==2]

original_particles.shape

In [ ]:
categories = ['CS', 'MC', 'SS']

data_directory = '/content/gan-generated/'
# categories = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

gan_data = []

def load_generated_data():
    for category in categories:
        path = os.path.join(data_directory, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_array = cv2.resize(img_array, (48,48))
                gan_data.append([resized_array, class_num])
            except Exception as e:
                pass

            
load_generated_data()
print(len(gan_data))

In [ ]:
import random
random.shuffle(gan_data)

x_gan = []
y_gan = []

for features,label in gan_data: 
    x_gan.append(features)
    y_gan.append(label)
    
x_gan = np.array(x_gan).reshape(-1, 48, 48, 1)
y_gan = np.array(y_gan).reshape(-1)

In [ ]:
gan_particles = x_gan
gan_CS = x_gan[y_gan==0]
gan_MC = x_gan[y_gan==1]
gan_SS = x_gan[y_gan==2]

In [ ]:
img1 = original_particles
img2 = gan_particles

In [ ]:
def image_scaling(images, inception_shape):
    images_list = list()
    for image in images:
        scaled_image = resize(image, inception_shape, 0)
        images_list.append(scaled_image)
    return np.asarray(images_list)


def image_preprocessing(images):
    images = images.astype('float32')
    images = image_scaling(images, (75,75,3))
    images = preprocess_input(images)
    return images
    

def fid_score(images1, images2):
            
    inception = InceptionV3(include_top= False, pooling='avg', input_shape=(75,75,3))
    
    preprocessed_images1 = image_preprocessing(images1)
    preprocessed_images2 = image_preprocessing(images2)
    
    features1 = inception.predict(preprocessed_images1)
    features2 = inception.predict(preprocessed_images2)
    
    m1 = features1.mean(axis=0)
    m2 = features2.mean(axis=0) 
    sigma1 = np.cov(features1, rowvar=False)
    sigma2 = np.cov(features2, rowvar=False)

    convmult = sqrtm(sigma1.dot(sigma2))
    
    if np.iscomplexobj(convmult):
        convmult = convmult.real
        
    fid = np.sum((m1 - m2)**2.0) + np.trace(sigma1 + sigma2 - 2.0 * convmult)
    return fid

In [ ]:
fid_score(img1, img2)

In [ ]:
#FID-scores

#particles-2500
#original-cvae: 147.84
#original-cgan: 135.62
#original-lsgan: 132.58
#original-infogan: 125.82

#---------------------------------

#fashion-2500
#original-cvae: 278.10
#original-cgan: 77.42     
#original-lsgan: 73.18    
#original-infogan: 70.77  